In [1]:
pip install requests pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import pandas as pd
import base64
import os
import tqdm

In [12]:
# Configuration
ROBOFLOW_API_KEY = "gTF1TpI6ZRLe9RyVr7gr"
ROBOFLOW_PROJECT = "aiml-cv-food-101-dectection"

In [13]:
# Utility function to generate Annoatation

# Upload image to Roboflow
def process_image(image_path):
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")
    
    url = f"https://detect.roboflow.com/{ROBOFLOW_PROJECT}/1?api_key={ROBOFLOW_API_KEY}"
    response = requests.post(url, data=image_data, headers={"Content-Type": "application/x-www-form-urlencoded"})

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

# Parse annotation and write CSV
def write_annotations_to_csv(response_json, output_csv="Food_101/annotations.csv"):
    predictions = response_json.get("predictions", [])

    rows = []
    for pred in predictions:
        rows.append({
            "class": pred.get("class", "N/A"),
            "x": pred.get("x"),
            "y": pred.get("y"),
            "width": pred.get("width"),
            "height": pred.get("height"),
            "confidence": pred.get("confidence")
        })

    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False)
    print(f"Annotations saved to {output_csv}")

In [14]:
from tqdm import tqdm  # Make sure this import exists
image_dir = "Food_101/"
annotate_details = []
try:
    for root, _, filenames in os.walk(image_dir):
        for fname in tqdm(filenames, desc="Processing Images"):
            full_path = os.path.join(root, fname)
            result = process_image(full_path)
            write_annotations_to_csv(result)
except Exception as e:
    print(f"Failed: {e}")


Processing Images:   0%|          | 0/1 [00:04<?, ?it/s]

Failed: Error: 403, {"message":"Forbidden"}
